<a href="https://colab.research.google.com/github/lytvyniuk/iowa_liquor_sales-Exploratory-Data-Analysis-/blob/master/Test_task_iowa_liquor_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# please run this cell and follow the link to authenticate
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

The dataset analysis can be useful to find out the best time to sell items, the most profitable vendors, the most popular drinks etc. This information eventually can help to understand businesses how to act to gain more revenue.

I start with general analysis of the data and go in details if I find an interesting observation.

**Data cleaning**

In the beginning, there were not any issues visible in the dataset to fix. During the analysis, when I found some issues so I made some corrections on the go. (Example of incorrect data -  codes for the same vendor , one is 65, second is 065). Ideally, they should be fixed first, maybe on table level in BigQuery also.

**Top 10 stores with the most bottles sold**

Let's find out what vendors are the biggest on the market in terms of sold bottles. There are a lot of them, so I showed top 10 on the plot for better visualisation.

In [0]:
# select aggregated data on bottles_sold and vendors
%%bigquery --project protean-genius-271221 df
SELECT vendor_number, sum(bottles_sold)  as bottles_sold FROM `bigquery-public-data.iowa_liquor_sales.sales`  group by vendor_number order by bottles_sold desc limit 10

In [0]:
# select unique names of the stores , because some stores have same name code but slight difference in names (uppercase and lowercase etc), so it is needed to group correctly
%%bigquery --project protean-genius-271221 df_names
SELECT distinct vendor_number, vendor_name FROM `bigquery-public-data.iowa_liquor_sales.sales` 

In [0]:
# merging with names
df = pd.merge(df, df_names, on = "vendor_number", how='left').drop_duplicates(subset='vendor_number', keep="first")
df


In [0]:
sns.barplot(x = 'vendor_name', y = 'bottles_sold',  data = df, color="green")
plt.xticks(rotation=50, horizontalalignment='right')
plt.xlabel("Vendor name")
plt.ylabel("Bottles sold")
plt.title("Top 10 biggest vendors")
plt.show()

**What is the most popular category of liquors in terms of bottles sold?**

In [0]:
%%bigquery --project protean-genius-271221 df
SELECT category,category_name, sum(bottles_sold) as bottles_sold FROM `bigquery-public-data.iowa_liquor_sales.sales`group by category,category_name order by bottles_sold desc limit 10

In [0]:
df

One category has same name (with different case) but different category code, which is strange. I leave it as it is, but if one of codes is wrong , 'Spiced Rum' would have a higher position.

In [0]:
sns.barplot(x = 'category_name', y = 'bottles_sold',  data = df, color="lightblue")
plt.xticks(rotation=50, horizontalalignment='right')
plt.xlabel("Vendor name")
plt.ylabel("Bottles sold")
plt.title("Top 10 most popuar categories of liquors ")
plt.show()

Vodka is the most popular one, American vodkas category in particular.




 **Let's see how many drinks are sold in different days, monthes, years for all vendors.**

In [0]:
%%bigquery --project protean-genius-271221 df_dates
SELECT sum(bottles_sold) as bottles_sold, FORMAT_DATE('%a',date) AS weekday FROM `bigquery-public-data.iowa_liquor_sales.sales` group by weekday

In [0]:
df_dates.dtypes

In [0]:
order = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", 'Sun']
sns.barplot(x = 'weekday', y = 'bottles_sold',  data = df_dates, color="blue", order = order)
plt.xlabel('Day of the week')
plt.ylabel('Bottles sold')
plt.title("Number of bottles sold by day of the week")
plt.show()



Interesting observation that liquors almost were not sold on weekends, which is most likely caused by law restrictions. However, there is small amount of sales on Saturday. Amount of bottles on Mon - Thu is almost on the same level but it is suprisingly higher comparing to sales on Friday.

In [0]:
# let's see something from Saturday data, where sales are much lower
%%bigquery --project protean-genius-271221 df
select * from (SELECT vendor_number, sum(bottles_sold) as bottles_sold, FORMAT_DATE('%a',date) AS weekday FROM `bigquery-public-data.iowa_liquor_sales.sales`group by vendor_number, weekday) where weekday = 'Sat' order by bottles_sold desc limit 10

In [0]:
# merging with names
df = pd.merge(df, df_names, on = "vendor_number", how='left').drop_duplicates(subset='vendor_number', keep="first")
df

In [0]:
sns.barplot(x = 'vendor_name', y = 'bottles_sold',  data = df, color="green")
plt.xticks(rotation=50, horizontalalignment='right')
plt.xlabel("Vendor name")
plt.ylabel("Bottles sold")
plt.title("Top biggest vendors on Saturday")
plt.show()

Results are same as from whole dataset as expected.

Also we can find out the most popular drink type.

In [0]:
# find out most popular types of liquor Saturday data
%%bigquery --project protean-genius-271221 df
select * from (SELECT category,category_name, sum(bottles_sold) as bottles_sold, FORMAT_DATE('%a',date) AS weekday FROM `bigquery-public-data.iowa_liquor_sales.sales`group by category,category_name, weekday) where weekday = 'Sat' order by bottles_sold desc limit 10

In [0]:
df

In [0]:
sns.barplot(x = 'category_name', y = 'bottles_sold',  data = df, color="lightblue")
plt.xticks(rotation=50, horizontalalignment='right')
plt.xlabel("Vendor name")
plt.ylabel("Bottles sold")
plt.title("Top 10 most popuar categories of liquors on Saturday")
plt.show()

Also more or less same result, but different category name is on top.

In [0]:
# group by month
%%bigquery --project protean-genius-271221 df_dates
SELECT sum(bottles_sold) as bottles_sold, FORMAT_DATE('%m',date) AS month FROM `bigquery-public-data.iowa_liquor_sales.sales` group by month order by month

In [0]:
df_dates

In [0]:
sns.barplot(x = 'month', y = 'bottles_sold',  data = df_dates, color="blue")
plt.xlabel('Month')
plt.ylabel('Bottles sold')
plt.title("Number of bottles sold by month")
plt.show()

The data shows that the busiest sales happen on October and December, which is surely related to upcoming holidays in these months (at least in December). Other months level of sales is lower without any significant differences.

In [0]:
# group by year
%%bigquery --project protean-genius-271221 df_dates
SELECT sum(bottles_sold) as bottles_sold, FORMAT_DATE('%Y',date) AS year FROM `bigquery-public-data.iowa_liquor_sales.sales` group by year order by year

In [0]:
df_dates

In [0]:
sns.lineplot(x = 'year', y = 'bottles_sold',  data = df_dates, color="blue")
plt.xlabel('Year')
plt.ylabel('Bottles sold')
plt.title("Number of bottles sold by year")
plt.show()

Sales are increasing every year ( for 2020 there is no full data yet).

**Biggest vendors in terms of income**

In [0]:
# select aggregated data on sale_dollars and vendors
# found same vendors with wrongly inserted codes. removing extra '0' in front of the number, so that codes are consistent
%%bigquery --project protean-genius-271221 df
SELECT replace(ltrim(replace(vendor_number,'0',' ')),' ','0') as vendor_number, sum(sale_dollars) as total_dollars FROM `bigquery-public-data.iowa_liquor_sales.sales`  group by vendor_number order by total_dollars desc limit 10

In [0]:
# merging with names
df = pd.merge(df, df_names, on = "vendor_number", how='left').drop_duplicates(subset='vendor_number', keep="first")
df

DIAGEO AMERICAS is on the first place by income and by bottles sold, other vendors positions differ by these attributes (income and by bottles sold).

In [0]:
top_vendors_by_doll = list(df['vendor_name'])
top_vendors_by_doll = list(map(str.upper,top_vendors_by_doll)) # easier way to make vendor names consistent is to make them all writtem in Uppercase or Lowercase assuming they don't have other issues
top_vendors_by_doll

In [0]:
# select aggregated data on sale_dollars and vendors .(easier way to make vendor names consistent is to make them all writtem in Uppercase or Lowercase assuming they don't have other issues)
%%bigquery --project protean-genius-271221 df
SELECT vendor_number, UPPER(vendor_name) as vendor_name, sum(sale_dollars) as total_dollars , FORMAT_DATE('%Y',date) AS year FROM `bigquery-public-data.iowa_liquor_sales.sales`  group by vendor_number,vendor_name, year order by total_dollars desc

In [0]:
df

In [0]:
df_vendors = df[df['vendor_name'].isin(top_vendors_by_doll)]
df_vendors = df_vendors[df_vendors['year'] != '2020'] # removing current year because it is not complete


In [0]:
df_vendors

In [0]:
g = sns.FacetGrid(df_vendors, col="vendor_name",col_wrap=5, aspect = 1.5)
g = g.map(sns.lineplot, 'year', 'total_dollars')
g.set_titles("{col_name} ")
g.set_xticklabels( rotation=30)

From the plots above, it can be observed that some vendors like JIM BEAM BRANDS, BACARDI U.S.A., INC., CONSTELLATION WINE COMPANY, INC., SAZERAC NORTH AMERICA have a decrease in their revenue in year 2016. This might be caused by some external reason in the country (how the elections in 2016 can be related or smth else?). In general, trends do not vary much for majority of vendors, for DIAGEO AMERICAS sales are significally increasing from year 2012.


Seems that for some companies data is missing from years 2015, 2016, that can cause wrong interpretation of results , so further analysis is needed according to the same daterange. Also, it can be due to name change of the vendor (or corrupted data) so missing years data belongs to new vendor name and is not displayed here. There can be other reasons as well, because I analysed small part of the vendors.
